In [24]:
import seaborn as sea
import matplotlib.pyplot as plt
import itertools
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn import svm, tree, neighbors, neural_network
from sklearn.feature_selection import SelectKBest, f_classif
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder


In [16]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

In [17]:
#Finding which categories need to have value replacements
train_data.columns[train_data.isna().any()].tolist()


['Age', 'Cabin', 'Embarked']

In [18]:
#adding some new features, most notably Alone and Title
data = [test_data, train_data]

for train_data in data:
    train_data.drop(columns=['Ticket', 'Cabin'], inplace=True)
    train_data.set_index(keys=['PassengerId'], drop=True, inplace=True)


    train_data['Alone'] = 1
    train_data['Alone'].loc[train_data['Parch'] + train_data['SibSp'] > 1] = 0

    train_nan_map = {'Age': train_data['Age'].mean(), 'Fare': train_data['Fare'].mean(
    ), 'Embarked': train_data['Embarked'].mode()[0]}

    train_data.fillna(value=train_nan_map, inplace=True)
    train_data['Title'] = train_data['Name'].str.split(
        ", ", expand=True)[1].str.split(".", expand=True)[0]
    train_data['Title'] = train_data['Title'].replace('Mlle', 'Miss')
    train_data['Title'] = train_data['Title'].replace('Ms', 'Miss')
    train_data['Title'] = train_data['Title'].replace('Mme', 'Mrs')
    
    stat_min = 10
    title_names = (train_data['Title'].value_counts() < stat_min)

    train_data['Title'] = train_data['Title'].apply(
        lambda x: 'Misc' if title_names.loc[x] == True else x)

    encoder = LabelEncoder()
    encode = ['Embarked', 'Sex', 'Title']
    for col in encode:
        train_data[col] = encoder.fit_transform(train_data[col])

X_train = train_data.loc[:, train_data.columns != 'Survived']
X_train = X_train.loc[:, X_train.columns != 'Name']

y_train = train_data.loc[:, 'Survived']

X_train, X_test, y_train, y_test = train_test_split(
    X_train, y_train, test_size=0.20, random_state=10)

In [21]:
#Let's try and make knn co-dominant
knn_clf = neighbors.KNeighborsClassifier()
knn_clf.fit(X_train.values, y_train.values)
print(knn_clf.score(X_test.values, y_test.values))
y_pred = knn_clf.predict(X_test.values)
y_truth = y_test.values



0.7206703910614525


In [25]:
test = SelectKBest(f_classif, k='all')
test_fit = test.fit(X_train, y_train)
feat_score = test_fit.scores_.round(3)
p_values = -np.log10(test_fit.pvalues_).round(3)

In [22]:
tn, fp, fn, tp = confusion_matrix(y_truth, y_pred).ravel()
print("Confusion Matrix")
print(confusion_matrix(y_truth, y_pred, labels=[0, 1]))
print("")
print("True Negatives", tn)
print("False Positives", fp)
print("False Negatives", fn)
print("True Positives", tp)
print ("False Positive rate", fp/(fp+tn))
print ("False Negative rate", fn/(fn+tp))


Confusion Matrix
[[95 22]
 [28 34]]

True Negatives 95
False Positives 22
False Negatives 28
True Positives 34
False Positive rate 0.18803418803418803
False Negative rate 0.45161290322580644
